In [36]:
import os
import sys
import gc

import pandas as pd
import numpy as np
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [37]:
# Files and Paths
files_ = {
    "historical_transactions": "data/historical_transactions.csv",
    "merchants": "data/merchants.csv",
    "new_merchant_transactions": "data/new_merchant_transactions.csv",
    "train": "data/train.csv",
    "test": "data/test.csv"
}

# Paths
binned_data_dir = "assembled_bins/"
model_data_dir = "model_input_bins/"

if not os.path.exists(model_data_dir):
    os.makedirs(model_data_dir)

# Card IDS
bins = list(set([x.split(".")[0][-6:] for x in os.listdir(binned_data_dir)]))
print(bins[:5])
bin_count = 1

['C_ID_6', 'C_ID_7', 'C_ID_4', 'C_ID_5', 'C_ID_2']


In [38]:
# Read In Bins
def read_in_bin_datasets(bin_id):
    """
    """
    df_train = pd.concat(
        objs=[pd.read_csv(binned_data_dir+x) for x in os.listdir(binned_data_dir) if 'train' in x and bin_id in x],
        axis=0
    )
    df_test = pd.concat(
        objs=[pd.read_csv(binned_data_dir+x) for x in os.listdir(binned_data_dir) if 'test' in x and bin_id in x],
        axis=0
    )
    df_hist_trans = pd.concat(
        objs=[pd.read_csv(binned_data_dir+x) for x in os.listdir(binned_data_dir) if 'hist' in x and bin_id in x],
        axis=0
    )
    df_new_trans = pd.concat(
        objs=[pd.read_csv(binned_data_dir+x) for x in os.listdir(binned_data_dir) if 'new_merch' in x and bin_id in x],
        axis=0
    )
    df_merchants = pd.read_csv(files_.get('merchants'), dtype=str)
    print("Tables Read In to Memory")
    return df_train, df_test, df_hist_trans, df_new_trans, df_merchants

# Feature Methods

In [118]:
def drop_err_targets(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Get all targets < -20
    train_ids = list(pd.Series.unique(train.loc[train['target'] < -20, :]['card_id']))
    
    # Drop these from transactional observations
    hist_trans = hist_trans.loc[hist_trans['card_id'].isin(train_ids), :]
    new_trans = new_trans.loc[new_trans['card_id'].isin(train_ids), :]
    
    return train, test, hist_trans, new_trans, merchants


def add_elapsed_months(train, test, hist_trans, new_trans, merchants):
    """
    Get
    """
    
    # Max First Active Month
    max_date = dt.datetime(year=2018, month=2, day=1)
    
    # Train
    train.loc[:, 'first_active_month'] = pd.to_datetime(train['first_active_month'])
    train['elapsed_time'] = (
        max_date - train['first_active_month']
    ).dt.days
    
    # Test
    test.loc[:, 'first_active_month'] = pd.to_datetime(test['first_active_month'])
    test.loc[:, 'elapsed_time'] = (
        max_date - test['first_active_month']
    ).dt.days
    
    return train, test, hist_trans, new_trans, merchants


def encode_auth_flag(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Historic Transactions
    hist_trans.loc[:, 'authorized_flag'] = hist_trans['authorized_flag'].map({'Y': 1, 'N': 0})
    assert all([x in [0, 1] for x in list(pd.Series.unique(hist_trans.authorized_flag))])
    
    # New Merchant Transactions
    new_trans.loc[:, 'authorized_flag'] = new_trans['authorized_flag'].map({'Y': 1, 'N': 0})
    assert all([x in [0, 1] for x in list(pd.Series.unique(new_trans.authorized_flag))])
    
    return train, test, hist_trans, new_trans, merchants


def add_authorized_rates_by_card(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    df_hist_agg = hist_trans.groupby(by=['card_id'], as_index=False).agg({'authorized_flag': np.mean})
    df_hist_agg.rename(columns={'authorized_flag': 'authorized_purchase_rate_hist'},
                      inplace=True)
    df_new_agg = new_trans.groupby(by=['card_id'], as_index=False).agg({'authorized_flag': np.mean})
    df_new_agg.rename(columns={'authorized_flag': 'authorized_purchase_rate_new'},
                     inplace=True)

    train = pd.merge(train, df_hist_agg, how='left', on=['card_id'])
    train = pd.merge(train, df_new_agg, how='left', on=['card_id'])
    test = pd.merge(test, df_hist_agg, how='left', on=['card_id'])
    test = pd.merge(test, df_new_agg, how='left', on=['card_id'])
    
    return train, test, hist_trans, new_trans, merchants


def installment_averages(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Historic Installment Summary Stats
    df_hist_agg = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments': [np.min, np.max, np.var]})
    df_hist_agg.columns = [x[0] if x[1] == '' else x[1] for x in df_hist_agg.columns]
    df_hist_agg.rename(columns={'amin': 'installments_min_hist',
                                'amax': 'installments_max_hist',
                                'var': 'installments_var_hist'},
                       inplace=True)
    
    # New Merch Installment Summary Stats
    df_new_agg = new_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments': [np.min, np.max, np.var]})
    df_new_agg.columns = [x[0] if x[1] == '' else x[1] for x in df_new_agg.columns]
    df_new_agg.rename(columns={'amin': 'installments_min_new',
                                'amax': 'installments_max_new',
                                'var': 'installments_var_new'},
                      inplace=True)
    
    # Train 
    train = pd.merge(train, df_hist_agg, how='left', on=['card_id'])
    train = pd.merge(train, df_new_agg, how='left', on=['card_id'])
    
    # Test
    test = pd.merge(test, df_hist_agg, how='left', on=['card_id'])
    test = pd.merge(test, df_new_agg, how='left', on=['card_id'])
    
    return train, test, hist_trans, new_trans, merchants
    

def installment_error_flag(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Assign types
    hist_trans.loc[:, 'installments'] = hist_trans['installments'].astype(float)
    new_trans.loc[:, 'installments'] = new_trans['installments'].astype(float)
    
    # 999 Counts
    hist_trans['installments_err_flag_999_hist'] = np.where(hist_trans['installments'].isin([999]), 1, 0)
    new_trans['installments_err_flag_999_new'] = np.where(new_trans['installments'].isin([999]), 1, 0)
    hist_trans['installments_err_flag_neg1_hist'] = np.where(hist_trans['installments'].isin([-1]), 1, 0)
    new_trans['installments_err_flag_neg1_new'] = np.where(new_trans['installments'].isin([-1]), 1, 0)
    hist_trans['installments_1_hist'] = np.where(hist_trans['installments'].isin([1]), 1, 0)
    new_trans['installments_1_new'] = np.where(new_trans['installments'].isin([1]), 1, 0)
    
    
    # Hist Agg
    # ---- ---- ----      ---- ---- ----
    # Installments sum, max
    df_hist_agg = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments': [np.sum, np.std]})
    df_hist_agg.columns = [x[0] if x[1] == '' else x[1] for x in df_hist_agg.columns]
    df_hist_agg.rename(columns={'sum': 'installments_sum_hist',
                                'std': 'installments_std_hist'},
                       inplace=True)
    
    # Installments 999 sum, std
    df_hist_agg_999 = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments_err_flag_999_hist': [np.sum, np.std]})
    df_hist_agg_999.columns = [x[0] if x[1] == '' else x[1] for x in df_hist_agg_999.columns]
    df_hist_agg_999.rename(columns={'sum': 'installments_err_flag_999_sum_hist',
                                    'std': 'installments_err_flag_999_std_hist'},
                       inplace=True)
    
    # Installments -1 sum, std
    df_hist_agg_neg1 = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments_err_flag_neg1_hist': [np.sum, np.std]})
    df_hist_agg_neg1.columns = [x[0] if x[1] == '' else x[1] for x in df_hist_agg_neg1.columns]
    df_hist_agg_neg1.rename(columns={'sum': 'installments_err_flag_neg1_sum_hist',
                                     'std': 'installments_err_flag_neg1_std_hist'},
                       inplace=True)
    
    # Installments 1 sum, std
    # Installments -1 sum, std
    df_hist_agg_pos1 = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments_1_hist': [np.sum, np.std]})
    df_hist_agg_pos1.columns = [x[0] if x[1] == '' else x[1] for x in df_hist_agg_pos1.columns]
    df_hist_agg_pos1.rename(columns={'sum': 'installments_1_sum_hist',
                                     'std': 'installments_1_std_hist'},
                       inplace=True)
    
    # New Agg
    # ---- ---- ----     ---- ---- ----
    # Installments sum, max
    df_new_agg = new_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments': [np.sum, np.std]})
    df_new_agg.columns = [x[0] if x[1] == '' else x[1] for x in df_new_agg.columns]
    df_new_agg.rename(columns={'sum': 'installments_sum_new',
                               'std': 'installments_std_new'},
                      inplace=True)
    
    # Installments 999 sum, std
    df_new_agg_999 = new_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments_err_flag_999_new': [np.sum, np.std]})
    df_new_agg_999.columns = [x[0] if x[1] == '' else x[1] for x in df_new_agg_999.columns]
    df_new_agg_999.rename(columns={'sum': 'installments_err_flag_999_sum_hist',
                                   'std': 'installments_err_flag_999_std_hist'},
                       inplace=True)
    
    # Installments -1 sum, std
    df_new_agg_neg1 = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments_err_flag_neg1_hist': [np.sum, np.std]})
    df_new_agg_neg1.columns = [x[0] if x[1] == '' else x[1] for x in df_new_agg_neg1.columns]
    df_new_agg_neg1.rename(columns={'sum': 'installments_err_flag_neg1_sum_hist',
                                     'std': 'installments_err_flag_neg1_std_hist'},
                       inplace=True)
    
    # Installments 1 sum, std
    df_new_agg_pos1 = hist_trans.groupby(by=['card_id'], as_index=False).\
        agg({'installments_1_hist': [np.sum, np.std]})
    df_new_agg_pos1.columns = [x[0] if x[1] == '' else x[1] for x in df_new_agg_pos1.columns]
    df_new_agg_pos1.rename(columns={'sum': 'installments_1_sum_hist',
                                     'std': 'installments_1_std_hist'},
                       inplace=True)
    
    # Merge Back
    # Train
    train = pd.merge(train, df_hist_agg, how='left', on=['card_id'])
    train = pd.merge(train, df_hist_agg_999, how='left', on=['card_id'])
    train = pd.merge(train, df_hist_agg_neg1, how='left', on=['card_id'])
    train = pd.merge(train, df_hist_agg_pos1, how='left', on=['card_id'])
    train = pd.merge(train, df_new_agg, how='left', on=['card_id'])
    train = pd.merge(train, df_new_agg_999, how='left', on=['card_id'])
    train = pd.merge(train, df_new_agg_neg1, how='left', on=['card_id'])
    train = pd.merge(train, df_new_agg_pos1, how='left', on=['card_id'])
    
    # Test
    test = pd.merge(test, df_hist_agg, how='left', on=['card_id'])
    test = pd.merge(test, df_hist_agg_999, how='left', on=['card_id'])
    test = pd.merge(test, df_hist_agg_neg1, how='left', on=['card_id'])
    test = pd.merge(test, df_hist_agg_pos1, how='left', on=['card_id'])
    test = pd.merge(test, df_new_agg, how='left', on=['card_id'])
    test = pd.merge(test, df_new_agg_999, how='left', on=['card_id'])
    test = pd.merge(test, df_new_agg_neg1, how='left', on=['card_id'])
    test = pd.merge(test, df_new_agg_pos1, how='left', on=['card_id'])
    
    return train, test, hist_trans, new_trans, merchants


def add_historic_sales_lags(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Aggregate
    df_hist_lag = hist_trans.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'avg_sales_lag3': np.mean,
           'avg_sales_lag6': np.mean,
           'avg_sales_lag12': np.mean})
    df_hist_lag.rename(columns={x: x+"_hist_trans" for x in list(df_hist_lag.columns) if x != 'card_id'},
                      inplace=True)
    df_new_lag = new_trans.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'avg_sales_lag3': np.mean,
           'avg_sales_lag6': np.mean,
           'avg_sales_lag12': np.mean})
    df_new_lag.rename(columns={ x: x+"_new_trans" for x in list(df_new_lag.columns) if x != 'card_id'},
                     inplace=True)
    
    # Final Merge
    train = pd.merge(train, df_hist_lag, how='left', on=['card_id'])
    train = pd.merge(train, df_new_lag, how='left', on=['card_id'])
    test = pd.merge(test, df_hist_lag, how='left', on=['card_id'])
    test = pd.merge(test, df_new_lag, how='left', on=['card_id'])
    
    return train, test, hist_trans, new_trans, merchants


def category_summary_stats(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Encode hist trans categories
    for val in list(pd.Series.unique(hist_trans.category_1)):
        hist_trans["category_1_{}_hist".format(str(val))] = np.where(hist_trans['category_1'] == val, 1, 0)
    # Category 2 1-5
    hist_trans['category_2'].fillna(99, inplace=True)
    for val in [1, 2, 3, 4, 5, 99]:
        hist_trans["category_2_{}_hist".format(str(int(val)))] = np.where(hist_trans['category_2'] == val, 1, 0)
    # category 3
    for val in list(pd.Series.unique(hist_trans.category_3)):
        hist_trans["category_3_{}_hist".format(str(val))] = np.where(hist_trans['category_3'] == val, 1, 0)
        
    # Encode new trans categories
    # Cat 1
    for val in list(pd.Series.unique(new_trans.category_1)):
        new_trans["category_1_{}_new".format(str(val))] = np.where(new_trans['category_1'] == val, 1, 0)
    # Category 2 1-5
    new_trans['category_2'].fillna(99, inplace=True)
    for val in [1, 2, 3, 4, 5, 99]:
        new_trans["category_2_{}_new".format(str(int(val)))] = np.where(new_trans['category_2'] == val, 1, 0)
    # Cat 3
    for val in list(pd.Series.unique(new_trans.category_3)):
        new_trans["category_3_{}_new".format(str(val))] = np.where(new_trans['category_3'] == val, 1, 0)
        
    # Hist Sum and standard Dev
    hist_trans_agg = hist_trans.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'category_1_Y_hist': {'category_1_Y_hist_std': np.std, 
                                 'category_1_Y_hist_sum': np.sum},
           'category_1_N_hist': {'category_1_N_hist_std': np.std, 
                                 'category_1_N_hist_sum': np.sum},
           'category_2_1_hist': {'category_2_1_hist_std': np.std,
                                 'category_2_1_hist_sum': np.sum},
           'category_2_2_hist': {'category_2_2_hist_std': np.std,
                                 'category_2_2_hist_sum': np.sum},
           'category_2_3_hist': {'category_2_3_hist_std': np.std,
                                 'category_2_3_hist_sum': np.sum},
           'category_2_4_hist': {'category_2_4_hist_std': np.std,
                                 'category_2_4_hist_sum': np.sum},
           'category_2_5_hist': {'category_2_5_hist_std': np.std,
                                 'category_2_5_hist_sum': np.sum},
           'category_2_99_hist': {'category_2_99_hist_std': np.std,
                                  'category_2_99_hist_sum': np.sum},
           'category_3_A_hist': {'category_3_A_hist_std': np.std, 
                                 'category_3_A_hist_sum': np.sum},
           'category_3_B_hist': {'category_3_B_hist_std': np.std, 
                                 'category_3_B_hist_sum': np.sum},
           'category_3_C_hist': {'category_3_C_hist_std': np.std, 
                                 'category_3_C_hist_sum': np.sum}
          })
    hist_trans_agg.columns = [x[0] if x[1] == "" else x[1] for x in hist_trans_agg.columns]
    
    # New Sum an standard dev
    new_trans_agg = new_trans.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'category_1_Y_new': {'category_1_Y_new_std': np.std, 
                                 'category_1_Y_new_sum': np.sum},
           'category_1_N_new': {'category_1_N_new_std': np.std, 
                                 'category_1_N_new_sum': np.sum},
           'category_2_1_new': {'category_2_1_new_std': np.std,
                                'category_2_1_new_sum': np.sum},
           'category_2_2_new': {'category_2_2_new_std': np.std,
                                'category_2_2_new_sum': np.sum},
           'category_2_3_new': {'category_2_3_new_std': np.std,
                                'category_2_3_new_sum': np.sum},
           'category_2_4_new': {'category_2_4_new_std': np.std,
                                'category_2_4_new_sum': np.sum},
           'category_2_5_new': {'category_2_5_new_std': np.std,
                                'category_2_5_new_sum': np.sum},
           'category_2_99_new': {'category_2_99_new_std': np.std,
                                 'category_2_99_new_sum': np.sum},
           'category_3_A_new': {'category_3_A_new_std': np.std, 
                                'category_3_A_new_sum': np.sum},
           'category_3_B_new': {'category_3_B_new_std': np.std, 
                                'category_3_B_new_sum': np.sum},
           'category_3_C_new': {'category_3_C_new_std': np.std, 
                                'category_3_C_new_sum': np.sum}
          })
    new_trans_agg.columns = [x[0] if x[1] == "" else x[1] for x in new_trans_agg.columns]
    
    # Get cols for fillna() later
    cols = [x for x in hist_trans_agg.columns if x != 'card_id']
    cols.extend([x for x in new_trans_agg.columns if x != 'card_id'])
    
    # Combine
    hist_trans_agg = pd.merge(hist_trans_agg, new_trans_agg, how='left', on=['card_id'])
    train = pd.merge(train, hist_trans_agg, how='left', on=['card_id'])
    test = pd.merge(test, hist_trans_agg, how='left', on=['card_id'])
    for col in cols:
        train[col].fillna(0, inplace=True)
        test[col].fillna(0, inplace=True)
    
    return train, test, hist_trans, new_trans, merchants


def month_lag_differences(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Prep Hist
    hist_month_diff = hist_trans.loc[:, ['card_id', 'purchase_date', 'month_lag']]
    hist_month_diff.loc[:, 'hist_trans_month_diff'] = (
        dt.datetime.today() - pd.to_datetime(hist_month_diff['purchase_date'])
    ).dt.days//30
    hist_month_diff['hist_trans_month_diff'] += hist_month_diff['month_lag']
    
    # Prep New
    new_month_diff = new_trans.loc[:, ['card_id', 'purchase_date', 'month_lag']]
    new_month_diff.loc[:, 'new_trans_month_diff'] = (
        dt.datetime.today() - pd.to_datetime(new_month_diff['purchase_date'])
    ).dt.days//30
    new_month_diff['new_trans_month_diff'] += new_month_diff['month_lag']
    
    # Aggregate Hist
    hist_month_diff = hist_month_diff.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'hist_trans_month_diff': [np.std, np.mean]})
    hist_month_diff.columns = [x[0] if x[1] == '' else x[0]+"_"+x[1] for x in hist_month_diff.columns]
    
    # Aggregate New
    new_month_diff = new_month_diff.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'new_trans_month_diff': [np.std, np.mean]})
    new_month_diff.columns = [x[0] if x[1] == '' else x[0]+"_"+x[1] for x in new_month_diff.columns]
    
    # Merge back
    train = pd.merge(train, hist_month_diff, how='left', on=['card_id'])
    train = pd.merge(train, new_month_diff, how='left', on=['card_id'])
    test = pd.merge(test, hist_month_diff, how='left', on=['card_id'])
    test = pd.merge(test, new_month_diff, how='left', on=['card_id'])
    
    # Return
    return train, test, hist_trans, new_trans, merchants


def min_max_purchase_dates(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Hist Calcualte
    hist_trans_stats = hist_trans.loc[:, ['card_id', 'purchase_date']]
    hist_trans_stats.loc[:, 'purchase_date'] = \
        (pd.DatetimeIndex(hist_trans_stats['purchase_date']).astype(np.int64) * 1e-9)
    
    # New Calculate
    new_trans_stats = new_trans.loc[:, ['card_id', 'purchase_date']]
    new_trans_stats.loc[:, 'purchase_date'] = \
        (pd.DatetimeIndex(new_trans_stats['purchase_date']).astype(np.int64) * 1e-9)
    
    # Hist Agg
    hist_trans_stats = hist_trans_stats.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'purchase_date': [np.ptp, 'max', 'min']})
    hist_trans_stats.columns = [x[0] if x[1] == '' else x[0]+"_"+x[1] for x in hist_trans_stats.columns]
    
    # New Agg
    new_trans_stats = new_trans_stats.groupby(
        by=['card_id'],
        as_index=False
    ).agg({'purchase_date': [np.ptp, 'max', 'min']})
    new_trans_stats.columns = [x[0] if x[1] == '' else x[0]+"_"+x[1] for x in new_trans_stats.columns]
    
    # Merge back
    train = pd.merge(train, hist_trans_stats, how='left', on=['card_id'], suffixes=['', '_hist'])
    train = pd.merge(train, new_trans_stats, how='left', on=['card_id'], suffixes=['', '_new'])
    test = pd.merge(test, hist_trans_stats, how='left', on=['card_id'], suffixes=['', '_hist'])
    test = pd.merge(test, new_trans_stats, how='left', on=['card_id'], suffixes=['', '_new'])
    
    return train, test, hist_trans, new_trans, merchants
    


def sales_lag_auth_interaction(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Sales Lag Features
    avg_sales_lags = ['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12']
    suffixes = ["_hist_trans", "_new_trans"]
    auth_purchase_rates = ["authorized_purchase_rate_hist", "authorized_purchase_rate_new"]
    
    # Interaction
    for col in avg_sales_lags:
        
        # Train
        train.loc[:, '{}_x_trans_auth_purchase_rate_hist'.format(col)] = (
            train[col+"_hist_trans"] * train['authorized_purchase_rate_hist']
        ).fillna(0)
        train.loc[:, '{}_x_trans_auth_purchase_rate_new'.format(col)] = (
            train[col+"_new_trans"] * train['authorized_purchase_rate_new']
        ).fillna(0)
        
        # Test
        test.loc[:, '{}_x_trans_auth_purchase_rate_hist'.format(col)] = (
            test[col+"_hist_trans"] * test['authorized_purchase_rate_hist']
        ).fillna(0)
        test.loc[:, '{}_x_trans_auth_purchase_rate_new'.format(col)] = (
            test[col+"_new_trans"] * test['authorized_purchase_rate_new']
        ).fillna(0)
    
    
    return train, test, hist_trans, new_trans, merchants
        
    

def pca_city_spread(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Prepare hist trans
    trans_hist_wide = hist_trans.pivot_table(
        index=['card_id'],
        columns=['city_id'],
        values=['merchant_id'],
        aggfunc='count'
    )
    trans_hist_wide.reset_index(inplace=True)
    trans_hist_wide.columns = [x[0] if x[1] == '' else "city_id_{}_trans_count".format(str(x[1])) 
                                for x in trans_hist_wide.columns]
    # FillNA
    hist_count_cols = [x for x in trans_hist_wide.columns if "trans_count" in x]
    for col in hist_count_cols:
        trans_hist_wide[col].fillna(0, inplace=True)
        
    # Prepate new trans
    trans_new_wide = new_trans.pivot_table(
        index=['card_id'],
        columns=['city_id'],
        values=['merchant_id'],
        aggfunc='count'
    )
    trans_new_wide.reset_index(inplace=True)
    trans_new_wide.columns = [x[0] if x[1] == '' else "city_id_{}_trans_count".format(str(x[1])) 
                                for x in trans_new_wide.columns]
    # FillNA
    new_count_cols = [x for x in trans_new_wide.columns if "trans_count" in x]
    for col in new_count_cols:
        trans_new_wide[col].fillna(0, inplace=True)
    
    # Establish PCA for hist trans
    pca = PCA(n_components=4)
    pca.fit_transform(trans_hist_wide[hist_count_cols])
    df_pcs = pca.components_.T
    df_pcs = pd.DataFrame(df_pcs[:,:4])
    df_pcs.columns = ['PC_0_city_id_hist_purchase_count', 'PC_1_city_id_hist_purchase_count',
                      'PC_2_city_id_hist_purchase_count', 'PC_3_city_id_hist_purchase_count']

    # Concenate and rebuild
    trans_hist_comp = pd.concat(
        objs=[trans_hist_wide[['card_id']], df_pcs],
        axis=1
    )
    
    # Establish PCA for new trans
    pca = PCA(n_components=4)
    pca.fit_transform(trans_new_wide[new_count_cols])
    df_pcs = pca.components_.T
    df_pcs = pd.DataFrame(df_pcs[:, :4])
    df_pcs.columns = ['PC_0_city_id_new_purchase_count', 'PC_1_city_id_new_purchase_count',
                      'PC_2_city_id_new_purchase_count', 'PC_3_city_id_new_purchase_count']
    
    # Concatenate new on to previously built
    trans_new_comp = pd.concat(
        objs=[trans_new_wide[['card_id']], df_pcs],
        axis=1
    )
    combined = pd.merge(trans_hist_comp, trans_new_comp, how='outer', on=['card_id'])
    
    # Merge on to train
    train = pd.merge(train, combined, how='left', on=['card_id'])
    test = pd.merge(test, combined, how='left', on=['card_id'])
    
    # Fill NA
    pca_cols = list(train.columns) + list(test.columns)
    pca_cols = [x for x in pca_cols if x[:3] == 'PC_']
    for col in pca_cols:
        if col in train.columns:
            train.loc[:, col] = train[col].fillna(0, inplace=False)
        if col in test.columns:
            test.loc[:, col] = test[col].fillna(0, inplace=False)
    
    # Add back to train and test
    return train, test, hist_trans, new_trans, merchants
    

def standard_scale_features(train, test, hist_trans, new_trans, merchants):
    """
    """
    
    # Cols to be dropped
    str_cols = ['first_active_month', 'card_id', 'binId']
    
    # Drop non features
    train.drop(labels=str_cols, axis=1, inplace=True)
    test.drop(labels=['first_active_month', 'binId'], axis=1, inplace=True)
    
    # Cols to have filled with na (for now)
    cols = [x for x in train.columns if x not in ['card_id', 'target']]
    for col in cols:
        train.loc[train[col].isnull(), col] = np.nanmedian(train[col])
        test.loc[test[col].isnull(), col] = np.nanmedian(test[col])
    
    # ---- ---- ----
    # Standard Scalar
    ss = StandardScaler()
    
    # Hold out card_id, target, and primary components
    col_names = [x for x in train.columns if (
        (x not in ['card_id', 'target'])
        and
        (x[:3] != "PC_")
    )]
    
    # Train
    features = train[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    train[col_names] = features
    
    # Test
    features = train[col_names]
    scaler.transform(features.values)
    test[col_names] = features
    
    return train, test, hist_trans, new_trans, merchants
    
    

# Run Feature Engineering

In [119]:
def main(bin_id):
    
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        read_in_bin_datasets(bin_id)

    # Drop Card IDs with Target less than -20
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        drop_err_targets(train=df_train,
                         test=df_test,
                         hist_trans=df_hist_trans,
                         new_trans=df_new_trans,
                         merchants=df_merch)

    # Add elapsed time from maximum first active date
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        add_elapsed_months(train=df_train, 
                           test=df_test, 
                           hist_trans=df_hist_trans, 
                           new_trans=df_new_trans,
                           merchants=df_merch)

    # Encode Authorized Flag in transaction tables
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        encode_auth_flag(train=df_train, 
                         test=df_test, 
                         hist_trans=df_hist_trans, 
                         new_trans=df_new_trans,
                         merchants=df_merch)

    # Add Authorized Purchase Rates
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        add_authorized_rates_by_card(train=df_train,
                                     test=df_test,
                                     hist_trans=df_hist_trans,
                                     new_trans=df_new_trans,
                                     merchants=df_merch)

    # Installment 999, -1, 1
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        installment_error_flag(train=df_train,
                             test=df_test,
                             hist_trans=df_hist_trans,
                             new_trans=df_new_trans,
                             merchants=df_merch)


    # Installment Averages
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        installment_averages(train=df_train, 
                             test=df_test, 
                             hist_trans=df_hist_trans, 
                             new_trans=df_new_trans,
                             merchants=df_merch)
    
    # Purchase Date Summary Stats
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        min_max_purchase_dates(train=df_train,
                               test=df_test,
                               hist_trans=df_hist_trans,
                               new_trans=df_new_trans,
                               merchants=df_merch)
    
    # Historic Sales Lags
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        add_historic_sales_lags(train=df_train, 
                                test=df_test, 
                                hist_trans=df_hist_trans, 
                                new_trans=df_new_trans,
                                merchants=df_merch)
    
    # Merchant Category Summary Stats
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        category_summary_stats(train=df_train,
                               test=df_test,
                               hist_trans=df_hist_trans,
                               new_trans=df_new_trans,
                               merchants=df_merch)
    
    # Purchase Month Lag Sumary Stats
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        month_lag_differences(train=df_train,
                              test=df_test,
                              hist_trans=df_hist_trans,
                              new_trans=df_new_trans,
                              merchants=df_merch)
    
    # Authorized Lag 
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        sales_lag_auth_interaction(train=df_train,
                                   test=df_test,
                                   hist_trans=df_hist_trans,
                                   new_trans=df_new_trans,
                                   merchants=df_merch)
    
    # PCA city_id
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        pca_city_spread(train=df_train, 
                        test=df_test, 
                        hist_trans=df_hist_trans, 
                        new_trans=df_new_trans,
                        merchants=df_merch)
    
    # Standard Scale Features
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = \
        standard_scale_features(train=df_train,
                                test=df_test,
                                hist_trans=df_hist_trans,
                                new_trans=df_new_trans,
                                merchants=df_merch)

    # Days after first active - purchased (purchase_date - first_active_date)
    
    # Save Prepped Datasets
    df_train.to_csv(model_data_dir + "train_{}_model_input.csv".format(str(bin_)), index=False)
    df_test.to_csv(model_data_dir + "test_{}_model_input.csv".format(str(bin_)), index=False)
    
    # Return
    return df_train, df_test, df_hist_trans, df_new_trans, df_merch


# ---- ---- ----      ---- ---- ----
# ---- ---- ----      ---- ---- ----
# TODO
# hist_trans and new_trans

# NEXT: indicators for category 2 - do not agg as floats, agg as indicators

#     hist purchase date max, new purchase date max
#     auth diff month, auth diff month lag
#     auth purchase rate diffs


#     aggregate category_1 - category_4
#     Differences between hist and new
#     weighted aggregate avg_purchases_lag3, 6, 12
#     weighted aggregate avg_sales_lag3, 6, 12


# DONE month_lag_diff from today to purchase date

# Save Prepped Datasets

In [120]:
for bin_ in bins:
    df_train, df_test, df_hist_trans, df_new_trans, df_merch = main(bin_id=bin_)
    print("Bin Complete :: {}".format(str(bin_)))
    
    
    

Tables Read In to Memory
Hist: 1
Hist: 2
Hist: 3
Hist: 4
Hist: 5
Hist: 99
New: 1
New: 2
New: 3
New: 4
New: 5
New: 99
           card_id     category_2_5_hist                        \
                   category_2_5_hist_std category_2_5_hist_sum   
0  C_ID_601956f203                   0.0                     0   
1  C_ID_604bdfae2d                   0.0                     0   
2  C_ID_60536a409e                   0.0                     0   
3  C_ID_6079b87ae3                   0.0                     0   
4  C_ID_60ad9fe219                   0.0                     0   

      category_2_99_hist                            category_2_1_hist  \
  category_2_99_hist_sum category_2_99_hist_std category_2_1_hist_std   
0                     54               0.482035              0.482035   
1                     17               0.411658              0.411658   
2                     10               0.363609              0.363609   
3                      0               0.000000        

Bin Complete :: C_ID_4
Tables Read In to Memory
Hist: 1
Hist: 2
Hist: 3
Hist: 4
Hist: 5
Hist: 99
New: 1
New: 2
New: 3
New: 4
New: 5
New: 99
           card_id     category_2_5_hist                        \
                   category_2_5_hist_std category_2_5_hist_sum   
0  C_ID_5007c783f0              0.000000                     0   
1  C_ID_502c380929              0.509902                    13   
2  C_ID_502ec01761              0.491793                    52   
3  C_ID_50608a83cc              0.000000                     0   
4  C_ID_506eaa15c5              0.188879                     5   

      category_2_99_hist                            category_2_1_hist  \
  category_2_99_hist_sum category_2_99_hist_std category_2_1_hist_std   
0                      1               0.117851              0.230669   
1                     11               0.503831              0.271746   
2                     31               0.482951              0.184561   
3                      8        

Bin Complete :: C_ID_3
Tables Read In to Memory
Hist: 1
Hist: 2
Hist: 3
Hist: 4
Hist: 5
Hist: 99
New: 1
New: 2
New: 3
New: 4
New: 5
New: 99
           card_id     category_2_5_hist                        \
                   category_2_5_hist_std category_2_5_hist_sum   
0  C_ID_001982552a              0.346547                    11   
1  C_ID_0029d52418              0.000000                     0   
2  C_ID_0046b2b355              0.000000                     0   
3  C_ID_0050fc55c0              0.000000                     0   
4  C_ID_005799985c              0.000000                     0   

      category_2_99_hist                            category_2_1_hist  \
  category_2_99_hist_sum category_2_99_hist_std category_2_1_hist_std   
0                      4               0.219320              0.392775   
1                    361               0.228228              0.228228   
2                      0               0.000000              0.000000   
3                      3        

Bin Complete :: C_ID_8
Tables Read In to Memory
Hist: 1
Hist: 2
Hist: 3
Hist: 4
Hist: 5
Hist: 99
New: 1
New: 2
New: 3
New: 4
New: 5
New: 99
           card_id     category_2_5_hist                        \
                   category_2_5_hist_std category_2_5_hist_sum   
0  C_ID_901d2fefab              0.000000                     0   
1  C_ID_902b1f1a4f              0.000000                     0   
2  C_ID_9071513607              0.000000                     0   
3  C_ID_909b4330be              0.000000                     0   
4  C_ID_909bc03536              0.083918                     1   

      category_2_99_hist                            category_2_1_hist  \
  category_2_99_hist_sum category_2_99_hist_std category_2_1_hist_std   
0                     17               0.427672              0.201229   
1                      9               0.447811              0.238833   
2                      4               0.206133              0.206133   
3                      9        

Bin Complete :: C_ID_d
Tables Read In to Memory
Hist: 1
Hist: 2
Hist: 3
Hist: 4
Hist: 5
Hist: 99
New: 1
New: 2
New: 3
New: 4
New: 5
New: 99
           card_id     category_2_5_hist                        \
                   category_2_5_hist_std category_2_5_hist_sum   
0  C_ID_e01e53b3d9              0.000000                     0   
1  C_ID_e04497f7e5              0.234049                   163   
2  C_ID_e06accca85              0.387553                    19   
3  C_ID_e071075f5a              0.000000                     0   
4  C_ID_e0782e0b15              0.401218                     8   

      category_2_99_hist                            category_2_1_hist  \
  category_2_99_hist_sum category_2_99_hist_std category_2_1_hist_std   
0                      0               0.000000              0.000000   
1                      0               0.000000              0.210620   
2                      4               0.387553              0.000000   
3                     19        

Bin Complete :: C_ID_c
Tables Read In to Memory
Hist: 1
Hist: 2
Hist: 3
Hist: 4
Hist: 5
Hist: 99
New: 1
New: 2
New: 3
New: 4
New: 5
New: 99
           card_id     category_2_5_hist                        \
                   category_2_5_hist_std category_2_5_hist_sum   
0  C_ID_a08630b5a1              0.142857                    48   
1  C_ID_a0b631321b              0.000000                     0   
2  C_ID_a0c16d8d3c              0.000000                     0   
3  C_ID_a0f81af158              0.000000                     0   
4  C_ID_a0f90270d3              0.000000                     0   

      category_2_99_hist                            category_2_1_hist  \
  category_2_99_hist_sum category_2_99_hist_std category_2_1_hist_std   
0                      0               0.000000              0.142857   
1                     11               0.510418              0.000000   
2                     27               0.000000              0.000000   
3                      0        